# 失敗時のハンドリング
タスクがなんらかの理由で失敗した場合のハンドリングについて試す。

In [1]:
# needs import
from prefect import Flow, task, Task
from prefect.engine.state import State
from datetime import timedelta

## リトライ
タスクは `max_retries` (最大リトライ回数)と `retry_delay` (リトライを行う間隔)を設定することができる。

これにより、タスクが失敗した際は指定された回数・間隔でリトライすることができる。

In [2]:
@task(max_retries=3, retry_delay=timedelta(seconds=5))
def retry_task():
    raise Exception("retry_task")

with Flow("retry") as flow_retry:
    retry_task()

flow_retry.run()

[2021-09-22 01:53:48+0900] INFO - prefect.FlowRunner | Beginning Flow run for 'retry'
[2021-09-22 01:53:48+0900] INFO - prefect.TaskRunner | Task 'retry_task': Starting task run...
[2021-09-22 01:53:48+0900] ERROR - prefect.TaskRunner | Task 'retry_task': Exception encountered during task execution!
Traceback (most recent call last):
  File "/Users/ryoasu/Library/Caches/pypoetry/virtualenvs/try-prefect-T2Jmh4JG-py3.9/lib/python3.9/site-packages/prefect/engine/task_runner.py", line 859, in get_task_run_state
    value = prefect.utilities.executors.run_task_with_timeout(
  File "/Users/ryoasu/Library/Caches/pypoetry/virtualenvs/try-prefect-T2Jmh4JG-py3.9/lib/python3.9/site-packages/prefect/utilities/executors.py", line 445, in run_task_with_timeout
    return task.run(*args, **kwargs)  # type: ignore
  File "/var/folders/6b/t3v4fdjs7gq6j0rw2gcb0s7r0000gn/T/ipykernel_2218/81358480.py", line 3, in retry_task
    raise Exception("retry_task")
Exception: retry_task
[2021-09-22 01:53:48+0900]

<Failed: "Some reference tasks failed.">

## 失敗時に呼ばれるcallback関数の設定
タスクが失敗した際に実行されるcallback関数を `@task(on_failure=<callback_fn>)` のように設定できる。

ただし、設定できるcallback関数は `fn(task: prefect.Task, state: prefect.engine.state.State) -> None` を満たす関数のみ。

In [3]:
def callback_fn(task: Task, state: State):
    print("📣 callback: ", task.name, state)

@task(on_failure=callback_fn)
def failed_task():
    raise Exception("failed_task")

with Flow("callback") as flow_callback:
    failed_task()

flow_callback.run()

[2021-09-22 01:54:03+0900] INFO - prefect.FlowRunner | Beginning Flow run for 'callback'
[2021-09-22 01:54:03+0900] INFO - prefect.TaskRunner | Task 'failed_task': Starting task run...
[2021-09-22 01:54:03+0900] ERROR - prefect.TaskRunner | Task 'failed_task': Exception encountered during task execution!
Traceback (most recent call last):
  File "/Users/ryoasu/Library/Caches/pypoetry/virtualenvs/try-prefect-T2Jmh4JG-py3.9/lib/python3.9/site-packages/prefect/engine/task_runner.py", line 859, in get_task_run_state
    value = prefect.utilities.executors.run_task_with_timeout(
  File "/Users/ryoasu/Library/Caches/pypoetry/virtualenvs/try-prefect-T2Jmh4JG-py3.9/lib/python3.9/site-packages/prefect/utilities/executors.py", line 445, in run_task_with_timeout
    return task.run(*args, **kwargs)  # type: ignore
  File "/var/folders/6b/t3v4fdjs7gq6j0rw2gcb0s7r0000gn/T/ipykernel_2218/2063309285.py", line 6, in failed_task
    raise Exception("failed_task")
Exception: failed_task
📣 callback:  fai

<Failed: "Some reference tasks failed.">

## タスクの状態遷移に応じたhandlerの設定
タスクの状態が遷移した際(e.g. Pending -> Runnning -> Failed)、実行されるstate handlerを `@task(state_handlers=[<callback_fn>])` のように設定できる。

ただし、設定できるhandlerは `fn(task: prefect.Task, old_state: prefect.engine.state.State, new_state: prefect.engine.state.State) -> None` を満たす関数のみ。

主に通知処理などに利用できる。

In [4]:
def handler(task: Task, old_state: State, new_state: State):
    print("📬 handler: ", task.name, old_state, new_state)

@task(state_handlers=[handler])
def failed_task():
    raise Exception("failed_task")

with Flow("callback") as flow_callback:
    failed_task()

flow_callback.run()

[2021-09-22 01:54:03+0900] INFO - prefect.FlowRunner | Beginning Flow run for 'callback'
[2021-09-22 01:54:03+0900] INFO - prefect.TaskRunner | Task 'failed_task': Starting task run...
📬 handler:  failed_task <Pending> <Running: "Starting task run.">
[2021-09-22 01:54:03+0900] ERROR - prefect.TaskRunner | Task 'failed_task': Exception encountered during task execution!
Traceback (most recent call last):
  File "/Users/ryoasu/Library/Caches/pypoetry/virtualenvs/try-prefect-T2Jmh4JG-py3.9/lib/python3.9/site-packages/prefect/engine/task_runner.py", line 859, in get_task_run_state
    value = prefect.utilities.executors.run_task_with_timeout(
  File "/Users/ryoasu/Library/Caches/pypoetry/virtualenvs/try-prefect-T2Jmh4JG-py3.9/lib/python3.9/site-packages/prefect/utilities/executors.py", line 445, in run_task_with_timeout
    return task.run(*args, **kwargs)  # type: ignore
  File "/var/folders/6b/t3v4fdjs7gq6j0rw2gcb0s7r0000gn/T/ipykernel_2218/1932910690.py", line 6, in failed_task
    rais

<Failed: "Some reference tasks failed.">